In [ ]:
import sys
sys.path.insert(0, '../toys-data')
import conf

In [1]:
import requests
import time
import random
import datetime

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import ssl
import smtplib
from bs4 import BeautifulSoup as bs

In [ ]:
project_id = '2024051800'

In [ ]:
giving_message = '한글테스트'

In [2]:
def o(*args):
    message = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '\t'
    for m in args:
        message += str(m)
        if m != args[-1]:
            message += '\t'
    print(message, file=sys.stderr)

In [3]:
def mail():
    # SMTP
    SMTP_SERVER = conf.SMTP_SERVER
    SMTP_PORT = conf.SMTP_PORT
    SMTP_USER = conf.SMTP_USER
    SMTP_PASSWORD = conf.SMTP_PASSWORD
    
    # mime
    msg = MIMEMultipart()
    msg['From'] = conf.SMTP_FROM
    msg['To'] = conf.SMTP_TO
    msg['Subject'] = giving_message

    mimetext = MIMEText(giving_message, _charset='utf-8')
    msg.attach(mimetext)

    # smtp
    # ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
    ssl_context = ssl.create_default_context()
    ssl_context.set_ciphers('DEFAULT:!DH')
    smtp = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT, context=ssl_context)
    smtp.login(SMTP_USER, SMTP_PASSWORD)
    smtp.sendmail(msg['From'], msg['To'], msg.as_string())
    smtp.close()

    o('[INFO] smtp done')

def slack():
    requests.post(
    f'https://hooks.slack.com/services/{conf.SLACK_WEBHOOK_URL}',
    headers={
        'content-type': 'application/json'
    },
    json={
        'text': giving_message,
        'blocks': [
        {
            'type': 'section',
            'text': {
            'type': 'mrkdwn',
            'text': giving_message
            }
        }
        ]
    }
    )
    o('[INFO] slack done')

In [4]:
def try_get_content():
    try:
        headers = conf.get_request_header(project_id)
        base_url = conf.get_base_url(project_id)
        data = conf.get_data(project_id)
        response = requests.post(base_url, headers=headers, data=data)
        return response.content
    except Exception as e:
        o('[EXCEPT] try_get_content')
        o('', e)
        return None
def try_get_remain_seat():
    try:
        content = try_get_content()
        soup = bs(content, "html.parser")
        seat_dic = {}
        for obj in soup.find_all('p', {'class':'na'}):
            text = obj.get_text()
            seat_alpha = text[0]
            if seat_alpha.startswith("방"):
                continue
            seat_number = int(text[1:])
            if seat_alpha not in seat_dic:
                seat_dic[seat_alpha] = []
            seat_disabled = 'disabled' not in obj.parent.parent.parent.parent.attrs['class']
            seat_dic[seat_alpha].append((seat_number, seat_disabled))
        #print(seat_dic)
        #print(seat_dic['A'])
        #print(seat_dic['A'][4:])
        result = ''
        for item in seat_dic['A'][4:]:
            if item[1]:
                result += str(item[0]) + ' '
        return result
    except Exception as e:
        o('[EXCEPT] try_get_remain_seat')
        o('', e)
        return None
def try_alarm():
    try:
        mail()
    except Exception as e:
        o('[EXCEPT] try_smtp')
        o('', e)
    try:
        slack()
    except Exception as e:
        o('[EXCEPT] try_slack')
        o('', e)
    #return
    import winsound
    import time
    frequency = 1500  # Set Frequency To 2500 Hertz
    duration = 1000  # Set Duration To 1000 ms == 1 second
    for _ in range(20):
        time.sleep(0.5)
        winsound.Beep(frequency, duration)


In [5]:
def check():
    remain_seat = try_get_remain_seat()
    if remain_seat == None:
        return False
    if len(remain_seat) > 0:
        o('A 자리', f'{remain_seat}')
        try_alarm(remain_seat)
    else:
        o('A 자리 없음')
    return True


In [ ]:
# try_alarm()
# check()
# slack()

In [6]:
while True:
    if check() == False:
        time.sleep(random.randrange(60 * 2, 60 * 3))
    else:
        time.sleep(random.randrange(50, 70))

[(1, True), (2, False), (3, False), (4, False), (5, False), (6, False), (7, False), (8, False), (9, False), (10, False), (11, False), (12, False)]


2024-05-18 13:30:19	A 자리 없음
